Notebook which will:
- Obtain alert photometry for named SN and create alert.
- Check whether this passes through a filter.

In [1]:
import requests, os
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.SimpleDecentFilter import SimpleDecentFilter
from ampel.contrib.hu.t0.PredetectionFilter import PredetectionFilter
from ampel.contrib.hu.t0.RedshiftCatalogFilter import RedshiftCatalogFilter

from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

import numpy as np

In [2]:
logger = AmpelLogger.get_logger()

In [3]:
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
token = os.environ["ARCHIVE_TOKEN"]

In [4]:
filename_ampel = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/test/190426c_ampel_cand.csv"
filename_kasliwal = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/test/190426c_kasliwal_cand.csv"

# Name of ZTF objects to lookup
snnames_ampel = np.genfromtxt(filename_ampel, delimiter=",", dtype=None,encoding="UTF-8")

snnames_kasliwal = np.genfromtxt(filename_kasliwal, delimiter=",", dtype=None,encoding="UTF-8")

print(snnames_ampel)
print(snnames_kasliwal)

trigger_jd = 2458600.14022394

['ZTF19aasknld' 'ZTF19aaslfre' 'ZTF19aaskmit' 'ZTF19aaslswv'
 'ZTF19aaslpds' 'ZTF19aaslzjf' 'ZTF19aatazmc' 'ZTF19aasldad'
 'ZTF19aaskiht' 'ZTF19aaslssx' 'ZTF19aasldsv' 'ZTF19aaslozu'
 'ZTF19aasmppj' 'ZTF19aaslxmg' 'ZTF19aaslpsk' 'ZTF19aaskpux'
 'ZTF19aasslob' 'ZTF19aaslfmk' 'ZTF19aaslovx' 'ZTF19aasrgik'
 'ZTF19aaslddj' 'ZTF19aaskikg' 'ZTF19aaskktr' 'ZTF19aataxum'
 'ZTF19aaskjyj' 'ZTF19aaskjsw' 'ZTF19aaskjwt' 'ZTF19aaskpbp'
 'ZTF19aaskmpt' 'ZTF19aasknfi' 'ZTF19aaslpqg' 'ZTF19aasmdqa'
 'ZTF19aaslell' 'ZTF19aaskihm' 'ZTF19aaskyxh' 'ZTF19aaslixg'
 'ZTF19aaskijn' 'ZTF19aasledp' 'ZTF19aaskipx' 'ZTF19aaslrms'
 'ZTF19aaslldo' 'ZTF19aasmqvm']
['ZTF19aasmftm' 'ZTF19aaslzjf' 'ZTF19aasmddt' 'ZTF19aasmekb'
 'ZTF19aassfws' 'ZTF19aaslszp' 'ZTF19aaslolf' 'ZTF19aaslozu'
 'ZTF19aasshpf' 'ZTF19aaslphi' 'ZTF19aaslpds' 'ZTF19aasmzqf'
 'ZTF19aaslzfk' 'ZTF19aaslvwn' 'ZTF19aasmdir']


In [5]:
def api_to_alert(name, token, tags=['FilterTest']):
    """
    Retrieve the alert history of a SN and return all recreated alerts.    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/alerts?with_history=false".format(name)

    header = {"Authorization": "bearer "+token}

    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    return [ZiAlertSupplier.shape_alert_dict( a, tags ) for a in response.json()]

In [6]:
def get_alert_array(snnames_array):
    alert_array = []
    for snname in snnames_array:
        alerts = api_to_alert(snname, token)
        #print(f'{snname}: Found {len(alerts)} alerts.')
        alert_array.append(alerts)
    return alert_array

In [7]:
alert_array_ampel = get_alert_array(snnames_ampel)
alert_array_kasliwal = get_alert_array(snnames_kasliwal)

Defining parameters for filter.

In [8]:
filter_config = {
    #"gaia_excessnoise_sig_max": 999,
    #"gaia_plx_signif": 3,
    #"gaia_pm_signif": 3,
    #"gaia_rs": 10,
    #"gaia_veto_gmag_max": 20,
    #"gaia_veto_gmag_min": 9,
    "min_ndet": 1, # should be 2
    "min_tspan": -1,
    "max_tspan": 365,
    #min_archive_tspan: -99
    "max_archive_tspan": 0.1,
    "min_drb": 0.3,
    "min_gal_lat": 0,
    "min_rb": 0.0,
    "min_sso_dist": 20,
    "ps1_confusion_rad": 3,
    "ps1_confusion_sg_tol": 0.1,
    "ps1_sgveto_rad": 1,
    "ps1_sgveto_th": 0.8,
    "max_fwhm": 5.5,
    # max_elong: 2
    "max_elong": 1.4,
    "max_magdiff": 1,
    "max_nbad": 2
}

predetection_filter_config = {
    "trigger_jd": trigger_jd,
    "gaia_excessnoise_sig_max": 999,
    "gaia_plx_signif": 3,
    "gaia_pm_signif": 3,
    "gaia_rs": 10,
    "gaia_veto_gmag_max": 20,
    "gaia_veto_gmag_min": 9,
    "min_ndet": 1,
    "min_tspan": -1,
    "max_tspan": 1,
    #"min_archive_tspan": -99,
    "max_archive_tspan": .1,
    "min_drb": 0.3,
    "min_gal_lat": 0,
    "min_rb": 0.0,
    "min_sso_dist": 20,
    "ps1_confusion_rad": 3,
    "ps1_confusion_sg_tol": 0.1,
    "ps1_sgveto_rad": 1,
    "ps1_sgveto_th": 0.8,
    "max_fwhm": 5.5,
    "max_elong": 2,
    "max_magdiff": 1,
    "max_nbad": 2,
    "resource": {"ampel-ztf/catalogmatch":"https://ampel.zeuthen.desy.de/api/catalogmatch/"} # gaia handling
}

In [9]:
t0filter = SimpleDecentFilter( **filter_config, logger=logger )
t0filter = PredetectionFilter( **predetection_filter_config, logger=logger )
t0filter.post_init()

In [21]:
# using SimpleDecentFilter
def simple_process(alerts):

    pass_array = []
    rejection_reason_array = []

    for alert in alerts:
        passes = True
        alert_jds = np.array(alert.get_values("jd", filters=t0filter.filter_pps))

        predetection_jds = [jd for jd in alert_jds if jd < trigger_jd]

        if len(predetection_jds) > 0:
            passes = False
            pass_array.append(passes)
            rejection_reason_array.append("predetection")
            continue



        foo = t0filter.process(alert)
        if not foo:
            #print(foo)
            passes = False
            pass_array.append(passes)
            rejection_reason_array.append("simpledecent")
            continue

        pass_array.append(passes)
        #print(foo)

    print("total:", np.any(pass_array), pass_array)
    #print("\t\t\t", rejection_reason_array)

In [10]:
#using PredetectionFilter
def process(alerts):

    pass_array = []
    rejection_reason_array = []

    for alert in alerts:
        passes = True

        foo = t0filter.process(alert)
        if not foo:
            #print(foo)
            passes = False
            #pass_array.append(passes)
            rejection_reason_array.append("decent")
            #continue

        pass_array.append(passes)
        #print(foo)

    #print("total:", np.any(pass_array), pass_array)
    return np.any(pass_array)
    #print("\t\t\t", rejection_reason_array)

In [11]:
def process_alerts(alert_array, name_array):
    pass_array = []
    for i, snname in enumerate(name_array):
        print(snname, end =" ")
        pass_alert = process(alert_array[i])
        
        pass_array.append(pass_alert)
        print(pass_alert)
    return pass_array


In [12]:
pass_ampel = process_alerts(alert_array_ampel, snnames_ampel)

ZTF19aasknld True
ZTF19aaslfre True
ZTF19aaskmit True
ZTF19aaslswv True
ZTF19aaslpds True
ZTF19aaslzjf True
ZTF19aatazmc True
ZTF19aasldad True
ZTF19aaskiht True
ZTF19aaslssx True
ZTF19aasldsv True
ZTF19aaslozu True
ZTF19aasmppj True
ZTF19aaslxmg True
ZTF19aaslpsk True
ZTF19aaskpux True
ZTF19aasslob True
ZTF19aaslfmk True
ZTF19aaslovx True
ZTF19aasrgik True
ZTF19aaslddj True
ZTF19aaskikg True
ZTF19aaskktr True
ZTF19aataxum True
ZTF19aaskjyj True
ZTF19aaskjsw True
ZTF19aaskjwt True
ZTF19aaskpbp True
ZTF19aaskmpt True
ZTF19aasknfi True
ZTF19aaslpqg True
ZTF19aasmdqa True
ZTF19aaslell True
ZTF19aaskihm True
ZTF19aaskyxh True
ZTF19aaslixg True
ZTF19aaskijn True
ZTF19aasledp True
ZTF19aaskipx True
ZTF19aaslrms True
ZTF19aaslldo True
ZTF19aasmqvm True


In [13]:
pass_kasliwal = process_alerts(alert_array_kasliwal, snnames_kasliwal)

ZTF19aasmftm False
ZTF19aaslzjf True
ZTF19aasmddt False
ZTF19aasmekb False
ZTF19aassfws False
ZTF19aaslszp False
ZTF19aaslolf False
ZTF19aaslozu True
ZTF19aasshpf False
ZTF19aaslphi False
ZTF19aaslpds True
ZTF19aasmzqf False
ZTF19aaslzfk True
ZTF19aaslvwn False
ZTF19aasmdir False


In [14]:
#print(np.isin(snnames_ampel, snnames_kasliwal))

mask = np.logical_and(np.logical_not(np.isin(snnames_kasliwal, snnames_ampel)), np.logical_not(pass_kasliwal))
kasliwal_only_filtered = snnames_kasliwal[mask]

print("Candidates in kasliwal that don't pass filter criteria:")
print(kasliwal_only_filtered)

Candidates in kasliwal that don't pass filter criteria:
['ZTF19aasmftm' 'ZTF19aasmddt' 'ZTF19aasmekb' 'ZTF19aassfws'
 'ZTF19aaslszp' 'ZTF19aaslolf' 'ZTF19aasshpf' 'ZTF19aaslphi'
 'ZTF19aasmzqf' 'ZTF19aaslvwn' 'ZTF19aasmdir']


In [ ]:
alert_array = get_alert_array(kasliwal_only_filtered)

process_alerts(alert_array, kasliwal_only_filtered)

In [80]:
alerts = alert_array[0]
for a in alerts:
    print(latest)
    latest = a.datapoints[0]
    # drb
    try:
        print('drb', latest['drb'])
    except KeyError:
        print('no drb')
    # rb
    try:
        print('rb', latest['rb'])
    except KeyError:
        print('no rb')
    # isdiffpos
    try:
        print('isdiffpos', latest['isdiffpos'])
    except KeyError:
        print('no isdiffpos')


{'jd': 2458601.9643056, 'fid': 2, 'pid': 847464303715, 'diffmaglim': 21.331493377685547, 'programid': 2, 'candid': 847464303715015104, 'isdiffpos': 't', 'tblid': 104, 'nid': 847, 'rcid': 37, 'field': 867, 'xpos': 1958.6419677734375, 'ypos': 1907.498779296875, 'ra': 325.9004479, 'dec': 77.8315634, 'magpsf': 20.937414169311523, 'sigmapsf': 0.15824905037879944, 'chipsf': 0.9810299873352051, 'magap': 20.756399154663086, 'sigmagap': 0.20589999854564667, 'distnr': 0.2524823546409607, 'magnr': 20.84000015258789, 'sigmagnr': 0.07699999958276749, 'chinr': 1.184000015258789, 'sharpnr': 0.03099999949336052, 'sky': 5.9032697677612305, 'magdiff': -0.18101400136947632, 'fwhm': 3.2699999809265137, 'classtar': 0.9819999933242798, 'mindtoedge': 1113.8580322265625, 'magfromlim': 0.5750929713249207, 'seeratio': 0.8667665719985962, 'aimage': 1.00600004196167, 'bimage': 0.8019999861717224, 'aimagerat': 0.3076452612876892, 'bimagerat': 0.24525994062423706, 'elong': 1.2543641328811646, 'nneg': 2, 'nbad': 0, 

In [53]:
alerts[0].datapoints

({'jd': 2458864.0659259,
  'fid': 2,
  'pid': 1109565924415,
  'diffmaglim': 19.72231674194336,
  'programid': 2,
  'candid': 1109565924415015004,
  'isdiffpos': 't',
  'tblid': 4,
  'nid': 1109,
  'rcid': 44,
  'field': 380,
  'xpos': 963.0435791015625,
  'ypos': 2300.962158203125,
  'ra': 250.1394082,
  'dec': -8.6740589,
  'magpsf': 19.557470321655273,
  'sigmapsf': 0.1663540005683899,
  'chipsf': 1.578667163848877,
  'magap': 19.33880043029785,
  'sigmagap': 0.2721000015735626,
  'distnr': 2.3860857486724854,
  'magnr': 22.381999969482422,
  'sigmagnr': 0.2280000001192093,
  'chinr': 0.7580000162124634,
  'sharpnr': -0.2029999941587448,
  'sky': 0.020566310733556747,
  'magdiff': -0.21866999566555023,
  'fwhm': 3.2799999713897705,
  'classtar': 0.9789999723434448,
  'mindtoedge': 779.5377807617188,
  'magfromlim': 0.383515864610672,
  'seeratio': 0.7646490931510925,
  'aimage': 0.8330000042915344,
  'bimage': 0.7390000224113464,
  'aimagerat': 0.2539634108543396,
  'bimagerat': 0.2

### Explore the RedshiftFilter

In [14]:
zfilter_config = {
    "catalog_match_radius": 30,
    "min_z": 0.002,
    "max_z": 0.025,
    "max_tspan": 5.,        # Max total detection age in alert
    "max_archive_tspan": 5.,        # Max total detection age in IPAC DB
    "min_rb": 0.3,             # real bogus score
    "min_drb": 0.995,         # deep learning real bogus score 
    # Should be set during proper install
    "resource": {"ampel-ztf/catalogmatch":"https://ampel.zeuthen.desy.de/api/catalogmatch/"},
}

In [15]:
zfilter = RedshiftCatalogFilter( **zfilter_config, logger=logger )
zfilter.post_init()

In [16]:
for alert in alerts:
    foo = zfilter.process(alert)
    print(alert.datapoints[0]['jd'], foo)

2458864.0659259 None
2459335.9309375 None
2459337.9111574 None
2459337.9413194 None
2459339.8768866 None
2459342.8762731 None
2459342.9180556 None
2459345.8988542 None
2459345.9414352 None
2460087.7539699 None
2460087.8678125 None
2460089.8064815 None
2460089.8441782 None


In [17]:
alerts[0].datapoints[0]['jd']

2458864.0659259